In [2]:
from fastai.vision import load_learner, Image, pil2tensor
from torch import no_grad as torch_no_grad
from cv2 import imread, split, merge
from detection_network import get_predictions

In [3]:
classes = ['10_p', '11_q', '12_r', '13_s', '13_s_rotated', '14_t', '15_e', '16_k', '17_u', '18_v', '19_w', '1_a', '20_f',
           '20_f_rotated', '21_g', '22_g', '23_h', '24_h', '25_f', '25_f_rotated', '26_f', '27_x', '27_x_rotated', '28_i',
           '28_i_rotated', '29_i', '2_a', '30_i', '30_i_rotated', '31_j', '32_j', '33_e', '34_k', '35_k', '36_i', 
           '36_i_rotated', '37_i', '38_l', '39_l', '3_b', '40_l', '41_y', '42_z', '42_z_rotated', '43_m', '44_m', 
           '44_m_rotated', '45_i', '45_i_rotated', '46_n', '46_n_rotated', '47_n', '47_n_rotated', '48_n', '48_n_rotated', 
           '49_aa', '4_b', '50_n', '51_n', '51_n_rotated', '52_ab', '53_ac', '54_ad', '55_l', '56_ae', '57_af', '58_i', 
           '59_o', '5_c', '60_o', '60_o_rotated', '6_ag', '6_ag_rotated', '7_c', '8_d', '9_d']

In [4]:
def get_prediction_image(path,sz=256):
    bgr_img = imread(path)
    b,g,r = split(bgr_img)
    rgb_img = merge([r,g,b])
    rgb_img = rgb_img/255
    img = Image(px=pil2tensor(rgb_img, np.float32))
    img = img.resize((3,sz,sz))
    return img.px.reshape(1,3,sz,sz)

In [5]:
learner = load_learner(path='.',file='pcb_msng_and_rotd.pkl',device='cpu')

In [154]:
model = learner.model.cpu()

In [187]:
img = get_prediction_image('/home/ubuntu/bmk_work/nnets/DLAssignment/24.24.jpg')

In [188]:
with torch_no_grad():
    output = model(img)

In [189]:
bboxes, preds, scores = get_predictions(output,0, detect_thresh=0.018)

In [190]:
pred_labels = [classes[pred] for pred in preds]

In [191]:
# pred_labels

In [192]:
rotated_comps = [pred for pred in pred_labels if 'rotated' in pred]
print(rotated_comps)

['27_x_rotated', '28_i_rotated']


In [186]:
with open("./DLAssignment/number_to_type.json") as read_file:
    json_str = read_file.read()

In [121]:
from ast import literal_eval

In [122]:
label_dict = literal_eval(json_str)

In [123]:
gt_labels = [k+'_'+v for k,v in label_dict.items()]

In [195]:
print(gt_labels)

['1_a', '2_a', '3_b', '4_b', '5_c', '6_ag', '7_c', '8_d', '9_d', '10_p', '11_q', '12_r', '13_s', '14_t', '15_e', '16_k', '17_u', '18_v', '19_w', '20_f', '21_g', '22_g', '23_h', '24_h', '25_f', '26_f', '27_x', '28_i', '29_i', '30_i', '31_j', '32_j', '33_e', '34_k', '35_k', '36_i', '37_i', '38_l', '39_l', '40_l', '41_y', '42_z', '43_m', '44_m', '45_i', '46_n', '47_n', '48_n', '49_aa', '50_n', '51_n', '52_ab', '53_ac', '54_ad', '55_l', '56_ae', '57_af', '58_i', '59_o', '60_o']


In [193]:
missing = set(gt_labels) - set(pred_labels)
rotated = set(pred_labels) - set(gt_labels)

In [194]:
for mc in missing:
    print(f"Component {mc.split('_')[0]} Missing")
for rot in rotated:
    print(f"Component {rot.split('_')[0]} Rotated")

Component 27 Rotated
Component 28 Rotated


In [198]:
missing.union(rotated)

{'27_x_rotated', '28_i_rotated'}